## Imports and download

In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
try:
    import google.colab
    IN_COLAB = True
    print("tyring to install stuff")
    ! pip install datasets transformers openai
    print("Running as a Colab notebook")
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")


tyring to install stuff
Running as a Colab notebook


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
import transformers

print(transformers.__version__)

4.30.2


In [ ]:
import torch
from torch.optim import AdamW
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.nn import MSELoss, Linear, TransformerEncoderLayer, LayerNorm, TransformerEncoder
from torch.optim import Adam
from torch.cuda.amp import GradScaler, autocast
import copy
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import openai
import numpy as np
import random

# Check if CUDA is available and choose device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_checkpoint = "distilgpt2"
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
from transformers import AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
model.transformer.h[1]

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [ ]:
def is_toeplitz(matrix):
    rows = len(matrix)
    cols = len(matrix[0])

    for i in range(rows):
        for j in range(cols):
            if i > 0 and j > 0 and matrix[i][j] != matrix[i-1][j-1]:
                return False

    return True
is_toeplitz(model.transformer.h[1].mlp.c_proj.weight)

False

In [ ]:
model.transformer.h[1].mlp

GPT2MLP(
  (c_fc): Conv1D()
  (c_proj): Conv1D()
  (act): NewGELUActivation()
  (dropout): Dropout(p=0.1, inplace=False)
)

## Hook into model and optimize sentence

In [ ]:
def unembed_and_decode(embeds_input):
  """
  Given an embedding vector, decode each token by using the transpose of the embedding matrix
  and grabbing the vocab token with the highest probability on each token.

  Also do this with the unembedding matrix as well.
  """
  with torch.no_grad():
      # Get the pre-trained embeddings
      pretrained_embeddings = model.transformer.wte.weight
      # Calculate dot product between input embeddings and pre-trained embeddings
      dot_product = torch.matmul(embeds_input, pretrained_embeddings.t())

      # Get the index of the highest value along dimension 2 (tokens)
      _, tokens = torch.max(dot_product, dim=2)

  # Decode tokens into text using the tokenizer
  text = tokenizer.batch_decode(tokens.tolist(), skip_special_tokens=True)

  return text

In [ ]:
def optimize_for_neuron(starting_sentence, layer_num=1, neuron_index=0, mlp_or_attention="mlp"):
  """
  Args:
    neuron_indices: List of indices.
    mlp_or_attention (str): 'mlp' or 'attention'
  """
  model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
  inputs = tokenizer(starting_sentence, return_tensors="pt").to(device)

  # Get embeddings
  with torch.no_grad():
      embeddings = model.transformer.wte(inputs["input_ids"])

  # Make embeddings require gradient
  embeddings.requires_grad_(True)

  # Create an optimizer for the embeddings
  optimizer = AdamW([embeddings], lr=0.1)  # You may need to adjust the learning rate
  pre_embeddings = embeddings.detach().clone()
  print(embeddings)
  print(unembed_and_decode(pre_embeddings))
  len_example = embeddings.shape[1] - 1

  if 'mlp' in mlp_or_attention:
    layer = model.transformer.h[layer_num].mlp
  else:
    raise NotImplementedError("Haven't implemented attention block yet")
  activation_saved = [torch.tensor(0.0)]
  def hook(model, input, output):
    # The output is a tensor. You can index it to get the activation of a specific neuron.
    # Here we're getting the activation of the 0th neuron.
    # TODO: Figure out what neruon this is actually grabbing. Why is it
    activation = output[0, len_example, neuron_index]
    activation_saved[0] = activation
  handle = layer.register_forward_hook(hook)

  losses = []
  dist = 0.0
  for i in tqdm(range(100)):
    outputs = model(inputs_embeds=embeddings, attention_mask=inputs.attention_mask)
    loss = -torch.sigmoid(activation_saved[0])
    loss.backward()
    optimizer.step()
    dist = torch.sum(embeddings - pre_embeddings).item()
    losses.append(loss)
    if i % 25 == 0:
      tqdm.write(f"\n{dist} and then {loss}\n")
      tqdm.write(unembed_and_decode(embeddings)[0])
    optimizer.zero_grad()

  return losses



In [ ]:
# input_sentence_1 = "In the midst of a vibrant summer morning, with the sun casting its golden rays upon the lush green meadows and the fragrant wildflowers swaying gently in the warm breeze, a multitude of birds chirped melodiously while gracefully soaring across the clear blue sky, their wings glimmering like tiny diamonds as they embraced the boundless freedom of the open air, and nearby, a majestic oak tree stood tall and proud, its branches extending outward in a magnificent display of nature's artistry, providing shade and shelter for a variety of creatures that sought solace beneath its protective canopy, including a family of squirrels playfully darting between the branches, their bushy tails serving as vibrant accents against the backdrop of verdant leaves, and as the day progressed, the distant rumble of thunder gradually grew louder, heralding the imminent arrival of a summer storm, as dark clouds gathered overhead, casting an ephemeral gloom over the once vibrant landscape, yet even in the face of this impending tempest, there was an undeniable beauty in the contrast between the electric flashes of lightning that briefly illuminated the sky and the cascading raindrops that danced upon the earth, breathing life into the thirsty soil and rejuvenating the flora and fauna, and as the storm subsided, a mesmerizing rainbow emerged, arching gracefully across the horizon, its vibrant hues painting a breathtaking scene that filled hearts with awe and wonder, reminding us of the ever-present magic and resilience of nature, and in that fleeting moment, as the world basked in the afterglow of the storm, a profound sense of gratitude and harmony washed over everything, reminding us of our intricate connection to the vast tapestry of existence."
# input_sentence_2 = "The fundamental principles of calculus provide a powerful framework for understanding and analyzing the rates of change and accumulation of quantities in various fields of mathematics and science, enabling us to model and solve complex real-world problems with precision and rigor."
# input_sentence_3 = "I'm sorry for the misunderstanding, but as an AI developed by OpenAI, I don't have direct access to individual sentences or documents from my training data. I was trained on a mixture of licensed data, data created by human trainers, and publicly available data. These sources may contain a wide range of data, including books, websites, and other texts, so I don't have the ability to recall or generate any specific sentence from the training data. I generate responses based on patterns and information in the data I was trained on."
# losses = optimize_for_neuron(input_sentence_3, neuron_index=2, layer_num=5)
# # Plot losses
# plt.figure(figsize=(10,6))
# plt.plot([loss.cpu().detach() for loss in losses])
# plt.title('Loss curve')
# plt.xlabel('Iteration')
# plt.ylabel('Loss')
# plt.grid(True)
# plt.show()

In [ ]:

def optimize_for_neuron_whole_input(neuron_index=0, layer_num=1, mlp_or_attention="mlp", num_tokens=10, num_iterations=200):
    """
    Args:
      neuron_indices: List of indices.
      mlp_or_attention (str): 'mlp' or 'attention'
    """
    model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)

    # Start with random embeddings
    embeddings = torch.randn((1, num_tokens, model.config.n_embd), device=device, requires_grad=True)

    # Create an optimizer for the embeddings
    optimizer = AdamW([embeddings], lr=0.1)  # You may need to adjust the learning rate

    if 'mlp' in mlp_or_attention:
        layer = model.transformer.h[layer_num].mlp
    else:
        raise NotImplementedError("Haven't implemented attention block yet")

    activation_saved = [torch.tensor(0.0, device=device)]
    def hook(model, input, output):
        # The output is a tensor. We're getting the average activation of the neuron across all tokens.
        activation = output[0, :, neuron_index].mean()
        activation_saved[0] = activation
    handle = layer.register_forward_hook(hook)

    losses = []
    for i in tqdm(range(num_iterations), position=0, leave=True):
        # Construct input for the model using the embeddings directly
        outputs = model(inputs_embeds=embeddings)
        # We want to maximize activation, which is equivalent to minimizing negative activation
        loss = -torch.sigmoid(activation_saved[0])
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        if i % (num_iterations//30) == 0:
            tqdm.write(f"Loss at step {i}: {loss.item()}\n", end='')
            tqdm.write(unembed_and_decode(embeddings)[0], end='')
        optimizer.zero_grad()

    handle.remove()  # Don't forget to remove the hook!
    return losses


In [ ]:
# losses = optimize_for_neuron_whole_input(neuron_index=2, layer_num=5, num_tokens=20)
# # Plot losses
# plt.figure(figsize=(10,6))
# plt.plot(losses)
# plt.title('Loss curve')
# plt.xlabel('Iteration')
# plt.ylabel('Loss')
# plt.grid(True)
# plt.show()


In [ ]:
class TransformerAutoencoder(torch.nn.Module):
    def __init__(self, model_checkpoint, latent_dim=100, nhead=2, num_layers=6):
        super().__init__()

        # Load the pretrained model
        base_model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

        # Create the encoder
        self.encoder = copy.deepcopy(base_model)
        self.encoder.lm_head = Linear(base_model.config.n_embd, latent_dim)

        # Create the decoder from scratch
        self.projection = Linear(latent_dim, base_model.config.n_embd)
        encoder_layer = TransformerEncoderLayer(d_model=base_model.config.n_embd, nhead=nhead)
        encoder_norm = LayerNorm(base_model.config.n_embd)
        self.decoder = TransformerEncoder(encoder_layer, num_layers=num_layers, norm=encoder_norm)

    def forward(self, input_ids, attention_mask=None):
        # Encode the input
        latent = self.encoder(input_ids, attention_mask=attention_mask).logits

        # Project the latent representation to the original embedding dimension
        projected = self.projection(latent)

        # Decode the projected representation
        reconstructed_embeddings = self.decoder(projected)

        return reconstructed_embeddings


In [ ]:
class Gpt2Autoencoder(torch.nn.Module):
    def __init__(self, model_checkpoint, latent_dim=10, nhead=2, num_layers=6):
        super().__init__()

        # Load the pretrained model
        base_model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

        # Create the encoder
        self.encoder = copy.deepcopy(base_model)
        self.encoder.lm_head = Linear(base_model.config.n_embd, latent_dim)

        # Create the decoder from scratch
        self.projection = Linear(latent_dim, base_model.config.n_embd)

        self.decoder = TransformerEncoder(encoder_layer, num_layers=num_layers, norm=encoder_norm)

    def forward(self, input_ids, attention_mask=None):
        # Encode the input
        latent = self.encoder(input_ids, attention_mask=attention_mask).logits

        # Project the latent representation to the original embedding dimension
        projected = self.projection(latent)

        # Decode the projected representation
        reconstructed_embeddings = self.decoder(projected)

        return reconstructed_embeddings


In [ ]:
def generate_sentence(model, tokenizer, max_length=50):
    # Randomly select a token
    random_token = random.randint(0, 50256)

    # Convert the token to a tensor
    input_ids = torch.tensor([[random_token]]).to(device)

    # Generate a sequence of tokens
    output = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=1.0, pad_token_id=tokenizer.eos_token_id)

    # Decode the tokens into a sentence
    sentence = tokenizer.decode(output[0], skip_special_tokens=True)
    return sentence

def train_autoencoder(autoencoder, criterion, optimizer, base_model, tokenizer, num_epochs=1000, print_every=50, save_path="transformer-autoencoder.pt", load_path=None):
    scaler = GradScaler()  # Enables mixed-precision training
    loss_values = []

    # Load the model's parameters from a checkpoint if provided
    if load_path is not None:
        autoencoder.load_state_dict(torch.load(load_path))

    # Training loop
    for epoch in tqdm(range(num_epochs)):
        # Generate a sentence with the pretrained model
        input_sentence = generate_sentence(base_model, tokenizer, max_length=50)

        # Prepare the inputs for the autoencoder
        input_ids = tokenizer.encode(input_sentence, return_tensors="pt").to(device)
        original_embeddings = base_model.transformer.wte(input_ids)

        # Run the autoencoder and compute the loss
        with autocast():
            reconstructed_embeddings = autoencoder(input_ids)
            loss = criterion(reconstructed_embeddings, original_embeddings)

        # Backpropagation
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Record the loss value for plotting
        loss_values.append(loss.item())

        # Print progress and save model every 'print_every' epochs
        if epoch % print_every == 0:
            print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item()}")
            torch.save(autoencoder.state_dict(), save_path)

    # Plot the loss values
    plt.figure(figsize=(10, 6))
    plt.plot(loss_values)
    plt.title('Loss Curve')
    plt.xlabel('Training Step')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.show()



In [ ]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

# Initialize autoencoder
autoencoder = TransformerAutoencoder('distilgpt2').to(device)

# Initialize loss function, optimizer, and gradient scaler for mixed-precision training
criterion = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(autoencoder.parameters())
scaler = GradScaler()

# Call the training function without a path to a checkpoint
# train_autoencoder(autoencoder, criterion, optimizer, model, tokenizer)


In [ ]:
autoencoder = TransformerAutoencoder('distilgpt2').to(device)
autoencoder.load_state_dict(torch.load('transformer-autoencoder.pt'))

<All keys matched successfully>

In [ ]:
# Encode the sentence
input_ids = tokenizer.encode("I am a banana", return_tensors="pt").to(device)
original_embeddings = model.transformer.wte(input_ids)
autoencoder(input_ids)

tensor([[[-0.9260, -1.3985,  0.9502,  ..., -0.9059,  0.5368,  0.5839],
         [-1.0884, -1.3361,  0.5918,  ..., -0.7432,  0.4180,  0.9456],
         [-0.9033, -1.3022,  0.8088,  ..., -0.8211,  0.2759,  0.7417],
         [-0.8096, -1.5392,  1.0889,  ..., -0.9813,  0.6752,  0.7932]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [ ]:

def get_sentence_similarity(sentence1, sentence2):
    # Get embeddings for both sentences
    response1 = openai.Embedding.create(input=sentence1, model="text-embedding-ada-002")
    response2 = openai.Embedding.create(input=sentence2, model="text-embedding-ada-002")

    embedding1 = np.array(response1['data'][0]['embedding'])
    embedding2 = np.array(response2['data'][0]['embedding'])

    # Compute cosine similarity between embeddings
    similarity = cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))

    return similarity[0][0]

def evaluate_autoencoder(autoencoder, base_model, tokenizer, num_samples=10):
    # Prepare the autoencoder for evaluation
    # autoencoder.eval() # <- do we need to implement this?

    similarities = []

    for _ in range(num_samples):
        # Generate a sentence with the base model
        input_sentence = generate_sentence(base_model, tokenizer, max_length=50)
        print(f"Original sentence: {input_sentence}")

        # Encode the sentence
        input_ids = tokenizer.encode(input_sentence, return_tensors="pt").to(device)
        original_embeddings = base_model.transformer.wte(input_ids)

        # Pass the encoded sentence through the autoencoder
        with torch.no_grad():
            reconstructed_embeddings = autoencoder(input_ids)

        # Decode the output of the autoencoder
        reconstructed_sentence = unembed_and_decode(reconstructed_embeddings)
        print(f"Reconstructed sentence: {reconstructed_sentence}")

        # Compute the sentence similarity between the original and reconstructed sentences
        similarity = get_sentence_similarity(input_sentence, reconstructed_sentence)
        similarities.append(similarity)

    # Compute the average sentence similarity
    average_similarity = np.mean(similarities)

    print(f"Average sentence similarity between original and reconstructed sentences: {average_similarity}")

# Call the evaluation function
evaluate_autoencoder(autoencoder, model, tokenizer)


Original sentence: uch the people of Israel would be able to have their way around what is perceived as a state and, ultimately, an absolute monarchy - a dictatorship not just to the one which has created the state. We saw this at the founding of the State of
Reconstructed sentence: [' horizont horizont horizont horizont horizont horizont horizont point horizont horizont horizont horizont horizont horizont horizont horizont point horizont horizont horizont horizontS point horizont point horizont horizont suspic point horizont point horizont suspic horizont horizont horizont suspic horizont horizont horizont point horizont horizont horizont horizont point horizont horizont point point']
Original sentence:  among it does the same thing in China. The only difference in the situation is that it has more people in China than the people in the United States, but in China the number increases so much. As a consequence for this, China is quite hard
Reconstructed sentence: [' suspic horizont ho

In [ ]:
def train_autoencoder(autoencoder, criterion, optimizer, base_model, tokenizer, num_epochs=1000, print_every=50, save_path="transformer-autoencoder.pt", load_path=None):
    scaler = GradScaler()  # Enables mixed-precision training
    loss_values = []
    similarity_values = []

    # Load the model's parameters from a checkpoint if provided
    if load_path is not None:
        autoencoder.load_state_dict(torch.load(load_path))

    # Training loop
    for epoch in tqdm(range(num_epochs)):
        # Generate a sentence with the pretrained model
        input_sentence = generate_sentence(base_model, tokenizer, max_length=50)

        # Prepare the inputs for the autoencoder
        input_ids = tokenizer.encode(input_sentence, return_tensors="pt").to(device)
        original_embeddings = base_model.transformer.wte(input_ids)

        # Run the autoencoder and compute the loss
        with autocast():
            reconstructed_embeddings = autoencoder(input_ids)
            loss = criterion(reconstructed_embeddings, original_embeddings)

        # Backpropagation
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Record the loss value for plotting
        loss_values.append(loss.item())

        # Compute the sentence similarity between the original and reconstructed sentences
        reconstructed_sentence = unembed_and_decode(reconstructed_embeddings)
        similarity = get_sentence_similarity(input_sentence, reconstructed_sentence)
        similarity_values.append(similarity)

        # Print progress and save model every 'print_every' epochs
        if epoch % print_every == 0:
            print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item()}, Similarity: {similarity}")
            torch.save(autoencoder.state_dict(), save_path)

    # Plot the loss values and similarities
    fig, ax1 = plt.subplots(figsize=(10, 6))

    color = 'tab:blue'
    ax1.set_xlabel('Training Step')
    ax1.set_ylabel('Loss', color=color)
    ax1.plot(loss_values, color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:red'
    ax2.set_ylabel('Similarity', color=color)  # we already handled the x-label with ax1
    ax2.plot(similarity_values, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()


In [ ]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

# Initialize autoencoder
autoencoder = TransformerAutoencoder('distilgpt2').to(device)

# Initialize loss function, optimizer, and gradient scaler for mixed-precision training
criterion = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(autoencoder.parameters())
scaler = GradScaler()


train_autoencoder(autoencoder, criterion, optimizer, model, tokenizer, load_path='transformer-autoencoder.pt')

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 0/1000, Loss: 0.014136187732219696, Similarity: 0.7101909368074204


  3%|▎         | 32/1000 [00:43<21:43,  1.35s/it]


KeyboardInterrupt: ignored

In [ ]:
def optimize_for_neuron_whole_input(neuron_index=0, layer_num=1, mlp_or_attention="mlp", num_tokens=10, num_iterations=200):
    """
    Args:
      neuron_indices: List of indices.
      mlp_or_attention (str): 'mlp' or 'attention'
    """
    base_model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
    autoencoder = TransformerAutoencoder('distilgpt2').to(device)
    autoencoder.load_state_dict(torch.load('transformer-autoencoder.pt'))

    # Get the dimensionality of the latent space
    latent_dim = autoencoder.encoder.lm_head.weight.shape[0]

    # Start with random latent vectors
    latent_vectors = torch.randn((1, num_tokens, latent_dim), device=device, requires_grad=True)

    # Create an optimizer for the latent vectors
    optimizer = AdamW([latent_vectors], lr=0.1)  # You may need to adjust the learning rate

    if 'mlp' in mlp_or_attention:
        layer = autoencoder.decoder.transformer.h[layer_num].mlp
    else:
        raise NotImplementedError("Haven't implemented attention block yet")

    activation_saved = [torch.tensor(0.0, device=device)]
    def hook(model, input, output):
        # The output is a tensor. We're getting the average activation of the neuron across all tokens.
        activation = output[0, :, neuron_index].mean()
        activation_saved[0] = activation
    handle = layer.register_forward_hook(hook)

    losses = []
    for i in tqdm(range(num_iterations), position=0, leave=True):
        # Construct input for the model using the embeddings directly
        embeddings = autoencoder.decoder(latent_vectors)
        outputs = base_model(inputs_embeds=embeddings)
        # We want to maximize activation, which is equivalent to minimizing negative activation
        loss = -torch.sigmoid(activation_saved[0])
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        if i % (num_iterations//30) == 0:
            tqdm.write(f"Loss at step {i}: {loss.item()}\n", end='')
            tqdm.write(unembed_and_decode(embeddings, tokenizer, base_model)[0], end='')
        optimizer.zero_grad()

    handle.remove()  # Don't forget to remove the hook!
    return losses


In [ ]:
optimize_for_neuron_whole_input()

AttributeError: ignored

torch.Size([10, 768])

In [ ]:
0